<a href="https://colab.research.google.com/github/settilea/Leandro_Setti_Proyecto_final_DataScience2/blob/main/LeandroSetti_Proyecto_Data_Science2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚖 NYC Taxi Trip Duration Analysis & Prediction
## 📋 Descripción del Proyecto
Este proyecto de Data Science tiene como objetivo predecir la duración total de viajes en taxi en la ciudad de Nueva York. Utilizando un enfoque de **Machine Learning** y **Análisis Geoespacial**, el modelo no solo considera variables temporales (hora/día), sino que enriquece los datos con información climática y características espaciales complejas (barrios y rutas teóricas).

El proyecto simula un escenario real de logística urbana donde la estimación precisa del tiempo de llegada (ETA) es crítica para la eficiencia operativa.

## 🎯 Objetivos
* Realizar un Análisis Exploratorio de Datos (EDA) profundo sobre el dataset de Taxis de NYC.
* Enriquecer el dataset original con datos externos: **Clima** y **Polígonos de Barrios (GeoJSON)**.
* Implementar ingeniería de características geoespaciales (Spatial Joins) para determinar zonas de origen y destino.
* Utilizar la **API de OSRM** para calcular rutas óptimas teóricas y comparar distancias de manejo vs. distancias euclidianas.
* Entrenar modelos de regresión (Linear, Random Forest, XGBoost) para predecir la duración del viaje.

## Instalación e importación de librerías necesarias para el proyecto

In [2]:
!pip install geopandas shapely polyline


In [4]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
from google.colab import userdata

# Configuración visual para ver todas las columnas
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore') # Para que no molesten los avisos de versiones

print("¡Librerías cargadas correctamente!")

¡Librerías cargadas correctamente!


## Carga del dataset
Cargamos el dataset directamente desde Kaggle, mediante la API

In [5]:
try:
    os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
    os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
    print("✅ Credenciales cargadas correctamente desde Secrets.")
except Exception as e:
    print("❌ Error: No se encontraron los secretos. Revisa el ícono de la Llave en la barra lateral.")

# 2. Descargar el dataset (Igual que antes, pero ahora "mágicamente" tiene permiso)
!kaggle datasets download -d yasserh/nyc-taxi-trip-duration

# 3. Descomprimir
!unzip -q -o nyc-taxi-trip-duration.zip  # -o para sobrescribir si ya existe

# 4. Verificar
import pandas as pd
print("\nArchivos listos:")
print(os.listdir())

✅ Credenciales cargadas correctamente desde Secrets.
Dataset URL: https://www.kaggle.com/datasets/yasserh/nyc-taxi-trip-duration
License(s): CC0-1.0
  0% 0.00/63.9M [00:00<?, ?B/s]
100% 63.9M/63.9M [00:00<00:00, 1.40GB/s]

Archivos listos:
['.config', 'NYC.csv', 'nyc-taxi-trip-duration.zip', 'sample_data']


## Análisis exploratorio del dataset
una vez extraidos los archivos de Kaggle vamos a realiza los primeros análisis para poder entender la información que tenemos

In [6]:
# 1. Cargar el archivo
print("Cargando dataset...")
df = pd.read_csv('NYC.csv')

# 2. Verificación rápida
print(f"¡Carga completa! El dataset tiene {df.shape[0]} filas y {df.shape[1]} columnas.")
display(df.head())

Cargando dataset...
¡Carga completa! El dataset tiene 1458644 filas y 11 columnas.


,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [7]:
# Convertir columnas de texto a fecha (Datetime), vital para poder sacar "hora del día", "día de la semana", etc.
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])

display(df.head())

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [8]:
# A. Ver tipos de datos y nulos
print("--- INFO GENERAL ---")
df.info()

# B. Estadísticas descriptivas
print("\n--- ESTADÍSTICAS ---")
# Usamos float_format para no ver notación científica (ej: 1.2e+05)
display(df.describe().apply(lambda s: s.apply('{0:.2f}'.format)))

--- INFO GENERAL ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   id                  1458644 non-null  object        
 1   vendor_id           1458644 non-null  int64         
 2   pickup_datetime     1458644 non-null  datetime64[ns]
 3   dropoff_datetime    1458644 non-null  datetime64[ns]
 4   passenger_count     1458644 non-null  int64         
 5   pickup_longitude    1458644 non-null  float64       
 6   pickup_latitude     1458644 non-null  float64       
 7   dropoff_longitude   1458644 non-null  float64       
 8   dropoff_latitude    1458644 non-null  float64       
 9   store_and_fwd_flag  1458644 non-null  object        
 10  trip_duration       1458644 non-null  int64         
dtypes: datetime64[ns](2), float64(4), int64(3), object(2)
memory usage: 122.4+ MB

--- ESTADÍSTICAS ---


,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
count,1458644.00,1458644.00,1458644.00,1458644.00,1458644.00,1458644.00,1458644.00,1458644.00,1458644.00
mean,1.53,.2f,.2f,1.66,-73.97,40.75,-73.97,40.75,959.49
min,1.00,.2f,.2f,0.00,-121.93,34.36,-121.93,32.18,1.00
25%,1.00,.2f,.2f,1.00,-73.99,40.74,-73.99,40.74,397.00
50%,2.00,.2f,.2f,1.00,-73.98,40.75,-73.98,40.75,662.00
75%,2.00,.2f,.2f,2.00,-73.97,40.77,-73.96,40.77,1075.00
max,2.00,.2f,.2f,9.00,-61.34,51.88,-61.34,43.92,3526282.00
std,0.50,nan,nan,1.31,0.07,0.03,0.07,0.04,5237.43


## Descripción de los atributos del dataset
###¿Qué representa cada variable del dataset?

- **id**: Un identificador único para cada viaje.
- **vendor_id**: Un código que indica el proveedor asociado con el registro del viaje.
- **pickup_datetime**: Fecha y hora en la que se activó el taxímetro.
- **dropoff_datetime**: Fecha y hora en la que se desactivó el taxímetro.
- **passenger_count**: El número de pasajeros en el vehículo (valor ingresado por el conductor).
- **pickup_longitude**: La longitud donde se activó el taxímetro.
- **pickup_latitude**: La latitud donde se activó el taxímetro.
- **dropoff_longitude**: La longitud donde se desactivó el taxímetro.
- **dropoff_latitude**: La latitud donde se desactivó el taxímetro.
- **store_and_fwd_flag**: Bandera que indica si el registro del viaje se mantuvo en la memoria del vehículo antes de enviarse al proveedor por falta de conexión (`Y` = sí, almacenado y enviado; `N` = no).